In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def MNIST_load():
    from keras.datasets import mnist
    (X, y), (_, _ )= mnist.load_data()
    X = X.reshape(-1, 784)
    X = (X > 128).astype(np.int32)
    y = y.astype(np.int32)
    return X, y

def one_hot_encode(y):
    num_classes = np.unique(y).shape[0]
    y_one_hot = np.zeros((y.shape[0], num_classes), dtype=int)
    y_one_hot[np.arange(y.shape[0]), y] = 1
    return y_one_hot

def train_test_split(X, y, split_ratio = 0.7):
    N = X.shape[0]
    rnd_idx = np.random.permutation(N)
    train_idx = rnd_idx[:int(split_ratio * X.shape[0])]
    test_idx = rnd_idx[int(split_ratio * X.shape[0]):]
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    y_test = y[test_idx]
    return X_train, y_train, X_test, y_test

In [4]:
X, y = MNIST_load()
X_train, y_train, X_test, y_test = train_test_split(X, y)
y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot = one_hot_encode(y_test)

In [61]:
X_train_small = X_train[:500]
y_train_small = y_train_one_hot[:500]
X_test_small = X_test[:20]
y_test_small = y_test[:20]

In [33]:
def AND(inputs):
    return np.all(inputs == 1).astype(int)

def OR(inputs):
    return np.any(inputs == 1).astype(int)

def XOR(inputs):
    return (np.sum(inputs) % 2).astype(int)

In [48]:
class autopoietic_ELM:
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def CA(self, X):
        X_ = np.copy(X)
        Nx, Ny = X.shape
        gates = [AND, OR, XOR]
        Φ = np.zeros((X.shape), dtype = int)
        gate = np.random.choice(gates, (X.shape)) #local
        ϵ = 2
        radius = 2.5

        for it in range(self.N_iter):
            print(f"Iteration {it + 1}/{self.N_iter}")

            x, y = np.indices(X.shape)
            new_state = np.zeros(X.shape)

            d_mask = np.sqrt((x - Nx//2)**2 + (y - Ny//2)**2) <= radius #distance mask 

            #state update
            for i in range(Nx):
                for j in range(Ny):
                    mask = np.roll(np.roll(d_mask, i - Nx//2, axis = 0), j - Ny//2, axis = 1)
                    new_state[i, j] = gate[i, j](X_[mask]) #local
            sync = (new_state == X_)
            Φ[sync] += 1 
            Φ[~sync] = 0

            mask_ensemble = (Φ == ε)
            X_ = new_state

            if np.any(mask_ensemble):
                ensemble_idxs = np.argwhere(mask_ensemble)
                for i, j in ensemble_idxs:
                    X_[np.roll(np.roll(d_mask, i - Nx//2, axis = 0), j - Ny//2, axis = 1)] = X_[i, j]

        return X_
        
    def train(self, N_iter):
        self.N_iter = N_iter
        X_ = self.CA(self.X)
        self.β = np.dot(np.linalg.pinv(X_), self.y)
            
    def predict(self, X):
        X_ = self.CA(X)
        X_ = np.dot(X_, self.β)
        pred = np.argmax(X_, axis = 1)
        return pred
        

In [62]:
net = autopoietic_ELM(X_train_small, y_train_small)

In [63]:
net.train(5)

Iteration 1/5
Iteration 2/5
Iteration 3/5
Iteration 4/5
Iteration 5/5


In [64]:
pred = net.predict(X_test_small)

Iteration 1/5
Iteration 2/5
Iteration 3/5
Iteration 4/5
Iteration 5/5


In [65]:
pred == y_test_small

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

Too computationally expensive to run. The non-linear projection with the CA that is.